<h1>Splice site analysis</h1>
<p>to do</p>.

In [ ]:
from fasta import FASTA
from csv import DictReader
from collections import defaultdict
import os

<h3>Data<.h3>


In [ ]:
hg38 = "/home/peter/data/hg38/GRCH_38.fa"

In [ ]:
exons = "exons_tab.txt"

In [ ]:
if not os.path.isfile(exons):
    raise FileNotFoundError(f"Could not find exons_tab")
if not os.path.isfile(hg38):
    raise FileNotFoundError(f"Could not find HG 38 genomes file at {hg38}")

In [ ]:
class Exon:

    def __init__(self, row) -> None:
        self.name = row["Name"]
        self.chr = row["Chr"]
        self.start = int(row["Start"])
        self.end = int(row["End"])
        if "+" in self.name:
            self.strand = "+"
        elif "-" in self.name:
            self.strand = "-"
        else:
            raise ValueError(f"Malformed name {self.name}")
        self.seq5 = None
        self.seq3 = None

           

    def chrom(self):
        return f"chr{self.chr}"

    def get_start(self):
        return self.start - 1

    def get_end(self):
        return self.end

    def get_strand(self):
        return self.strand

    def is_pos_strand(self):
        return self.strand == "+"
    
    def set_seq5(self, seq):
        self.seq5 = seq
        
    def set_seq3(self):
        self.seq3 = seq

In [ ]:
exon_list = []

with open(exons) as f:
    exon_names = set()
    reader = DictReader(f, delimiter="\t")
    for row in reader:
        if row['Type'] == 0:
            continue
        name = row["Name"]
        if name in exon_names:
            continue  # skip duplicates
        else:
            exon_names.add(name)
        exon_n = row["ExonNumber"]
        exon_count = row["NumExons"]
        print(name + ": " + exon_n +"/" + exon_count)
        exon_list.append(Exon(row=row))
print(f"Extracted {len(exon_list)} Exons")

<h3>Get splice site sequences</h3>

In [ ]:
fasta = FASTA(hg38)
fasta_ids = fasta.ids  # e.g., chr4
sequences = fasta.sequences  # BioPython SeqRecord objects

In [ ]:
# sort the target exons by chromosome
chrom_to_exonlist_d = defaultdict(list)
for exon in exon_list:
    chrom = exon.chrom()
    chrom_to_exonlist_d[chrom].append(exon)
print(f"Extracted exons on {len(chrom_to_exonlist_d)} chromosomes")

In [ ]:
for chromosome, exons in chrom_to_exonlist_d.items():
    chrom_sequence = sequences.get(chromosome)
    if chrom_sequence is None:
        raise ValueError(f"Could not retrieve sequence for {chromosome}")
    for exon in exons:
        if exon.is_pos_strand():
            print("is pos")
            # get 5 prime sequence
        else:
            print("is neg")
        